In [22]:
# spark.sql('drop table tablex').collect()
spark.sql('create table tablex (a string,id string,part_id string,stime int) stored as parquet')



In [21]:
spark.sql('insert into tablex values("A","user1","part1",1),("B","user1","part1",1),("C","user1","part2",1),("A","user1","part2",1),("A","user1","part2",1),("A","user1","part26",1),("D","user1","part24",1),("Z","user1","part42",1),("A","user1","part22",1),("A","user12","part2",1)')


In [1]:
# df.show()
df=spark.table('tablex')
df.show()

+---+-----+-------+-----+
|  a|   id|part_id|stime|
+---+-----+-------+-----+
|  D|user1| part24|    1|
|  Z|user1| part42|    1|
|  B|user1|  part1|    1|
|  C|user1|  part2|    1|
|  D|user1| part24|    1|
|  Z|user1| part42|    1|
|  A|user1|  part1|    1|
|  D|user1| part24|    1|
|  Z|user1| part42|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
+---+-----+-------+-----+
only showing top 20 rows



In [2]:
from pyspark.sql import functions as F
from pyspark.sql import types as T


In [3]:
df1=df.groupby('part_id','id').agg(F.collect_list("a").alias('ab') )
df1.show()

+-------+------+--------------------+
|part_id|    id|                  ab|
+-------+------+--------------------+
|  part2| user1|[C, A, A, C, A, A...|
| part26| user1|[A, A, A, A, A, A...|
|  part2|user12|[A, A, A, A, A, A...|
| part22| user1|[A, A, A, A, A, A...|
| part24| user1|[D, D, D, D, D, D...|
| part42| user1|[Z, Z, Z, Z, Z, Z...|
|  part1| user1|[B, A, B, A, B, A...|
+-------+------+--------------------+



# Output

In [4]:
# print(dir(F))
from pyspark.sql.functions import udf
from collections import Counter
import itertools

def prod(s):
    return dict(Counter(list(map(lambda x:x[0]+x[1],list(itertools.product(s,s))))))

prod1 = udf(prod)


df1.withColumn("ab", prod1("ab")).collect()


[Row(part_id='part2', id='user1', ab='{CC=4194304, AA=16777216, AC=8388608, CA=8388608}'),
 Row(part_id='part26', id='user1', ab='{AA=4194304}'),
 Row(part_id='part2', id='user12', ab='{AA=4194304}'),
 Row(part_id='part22', id='user1', ab='{AA=4194304}'),
 Row(part_id='part24', id='user1', ab='{DD=4194304}'),
 Row(part_id='part42', id='user1', ab='{ZZ=4194304}'),
 Row(part_id='part1', id='user1', ab='{AA=4194304, BB=4194304, AB=4194304, BA=4194304}')]

# Explain Plan

In [27]:
df1.withColumn("ab", prod1("ab")).explain()

== Physical Plan ==
*(2) Project [part_id#237, id#236, pythonUDF0#288 AS ab#284]
+- BatchEvalPython [prod(ab#261)], [part_id#237, id#236, ab#261, pythonUDF0#288]
   +- ObjectHashAggregate(keys=[part_id#237, id#236], functions=[collect_list(a#235, 0, 0)])
      +- Exchange hashpartitioning(part_id#237, id#236, 200)
         +- ObjectHashAggregate(keys=[part_id#237, id#236], functions=[partial_collect_list(a#235, 0, 0)])
            +- *(1) FileScan parquet default.tablex[a#235,id#236,part_id#237] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/home/arun/Spark/spark-warehouse/tablex], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<a:string,id:string,part_id:string>


In [28]:
spark.sql('insert into tablex select * from tablex')
spark.sql('select count(*) from tablex').collect()

[Row(count(1)=20480)]